# 🌊 Module 4: การวิเคราะห์คุณภาพน้ำด้วย Google Earth Engine
## Water Quality Analysis with GEE Python API

**Workshop: การใช้ GEE วิเคราะห์ข้อมูลอุตุ-อุทก-ไฟป่า-คุณภาพน้ำ**

---

### 📋 เนื้อหาในบทนี้
1. NDWI & MNDWI - ดัชนีน้ำ
2. Turbidity (ความขุ่น) จาก Sentinel-2
3. Chlorophyll-a Estimation (คลอโรฟิลล์-เอ)
4. TSS (Total Suspended Solids)
5. Sea Surface Temperature (SST)
6. การวิเคราะห์แหล่งน้ำจืด (เขื่อน/อ่างเก็บน้ำ)

### 📌 พื้นที่ศึกษา: อ่าวไทย, ทะเลสาบสงขลา, เขื่อนภูมิพล
### ⏱️ เวลา: ~35 นาที (11:20 - 11:55)

---
## 1. Setup & เชื่อมต่อ GEE

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# ee.Authenticate()
ee.Initialize(project='YOUR-PROJECT-ID')  # <-- ใส่ GEE Cloud Project ID

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['figure.dpi'] = 100
print('GEE เชื่อมต่อสำเร็จ ✅')

In [ ]:
# กำหนดพื้นที่ศึกษา
thailand = ee.FeatureCollection('FAO/GAUL/2015/level0') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

# พื้นที่ศึกษา 1: อ่าวไทยตอนบน
upper_gulf = ee.Geometry.Rectangle([99.5, 12.5, 101.5, 13.8])

# พื้นที่ศึกษา 2: ทะเลสาบสงขลา
songkhla_lake = ee.Geometry.Rectangle([100.1, 7.1, 100.6, 7.7])

# พื้นที่ศึกษา 3: เขื่อนภูมิพล
bhumibol_dam = ee.Geometry.Rectangle([98.9, 17.1, 99.2, 17.5])

Map = geemap.Map(center=[13.0, 100.5], zoom=6)
Map.addLayer(upper_gulf, {'color': 'blue'}, 'Upper Gulf of Thailand')
Map.addLayer(songkhla_lake, {'color': 'green'}, 'Songkhla Lake')
Map.addLayer(bhumibol_dam, {'color': 'red'}, 'Bhumibol Dam')
Map

---
## 2. Cloud Masking สำหรับ Sentinel-2

สำคัญมากสำหรับการวิเคราะห์น้ำ - เมฆจะทำให้ค่าผิดพลาด

In [ ]:
def mask_s2_clouds(image):
    """Mask clouds ด้วย SCL band ของ Sentinel-2"""
    scl = image.select('SCL')
    # SCL classes ที่ไม่ต้องการ: 3=cloud shadow, 8=cloud medium, 9=cloud high, 10=cirrus
    cloud_mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))
    return image.updateMask(cloud_mask)

def get_s2_clean(aoi, start_date, end_date, cloud_pct=20):
    """โหลด Sentinel-2 ที่ mask cloud แล้ว"""
    return ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_pct)) \
        .map(mask_s2_clouds) \
        .median()

print('ฟังก์ชัน Cloud Masking พร้อมใช้งาน ✅')

---
## 3. NDWI & MNDWI - ดัชนีน้ำ

**NDWI** (McFeeters, 1996)
$$NDWI = \frac{Green - NIR}{Green + NIR}$$

**MNDWI** (Xu, 2006) - Modified NDWI
$$MNDWI = \frac{Green - SWIR1}{Green + SWIR1}$$

MNDWI ดีกว่า NDWI ในพื้นที่เมืองเพราะลดผลกระทบจากสิ่งปลูกสร้าง

In [ ]:
# คำนวณ NDWI และ MNDWI สำหรับอ่าวไทยตอนบน
s2_gulf = get_s2_clean(upper_gulf, '2024-01-01', '2024-03-31')

# NDWI = (B3 - B8) / (B3 + B8)
ndwi = s2_gulf.normalizedDifference(['B3', 'B8']).rename('NDWI')

# MNDWI = (B3 - B11) / (B3 + B11)
mndwi = s2_gulf.normalizedDifference(['B3', 'B11']).rename('MNDWI')

vis_water_index = {'min': -0.5, 'max': 0.8, 'palette': ['brown', 'white', 'cyan', 'blue', 'navy']}

Map2 = geemap.Map(center=[13.2, 100.5], zoom=9)
Map2.addLayer(s2_gulf.clip(upper_gulf),
             {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000},
             'True Color')
Map2.addLayer(ndwi.clip(upper_gulf), vis_water_index, 'NDWI')
Map2.addLayer(mndwi.clip(upper_gulf), vis_water_index, 'MNDWI', shown=False)
Map2

---
## 4. Turbidity (ความขุ่นของน้ำ)

ประมาณค่าความขุ่นจาก Sentinel-2 โดยใช้สัดส่วนของ Band แดงกับ Band เขียว

**Turbidity Index** (Dogliotti et al., 2015)  
ใช้ Red band reflectance เป็นตัวชี้วัดหลัก  
ค่า Red reflectance สูง → น้ำขุ่นมาก

In [ ]:
# คำนวณ Turbidity Index
# วิธีง่าย: ใช้อัตราส่วน B4 (Red) / B3 (Green)
# น้ำขุ่น → Red สูง → Ratio สูง

# Normalized Difference Turbidity Index (NDTI)
# NDTI = (Red - Green) / (Red + Green)
ndti = s2_gulf.normalizedDifference(['B4', 'B3']).rename('NDTI')

# Simple Turbidity proxy using Red band (scaled)
turbidity_proxy = s2_gulf.select('B4').divide(10000).rename('Turbidity_Proxy')

# แสดงเฉพาะพื้นที่น้ำ (mask ด้วย NDWI > 0)
water_only = ndwi.gt(0)
ndti_water = ndti.updateMask(water_only).clip(upper_gulf)
turb_water = turbidity_proxy.updateMask(water_only).clip(upper_gulf)

vis_turb = {
    'min': -0.2,
    'max': 0.3,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'brown']
}

Map3 = geemap.Map(center=[13.2, 100.5], zoom=9)
Map3.addLayer(ndti_water, vis_turb, 'NDTI (Turbidity Index)')
Map3.add_colorbar(vis_turb, label='NDTI', layer_name='NDTI (Turbidity Index)')
Map3

---
## 5. Chlorophyll-a Estimation (คลอโรฟิลล์-เอ)

ตัวชี้วัดสาหร่ายในน้ำ - บ่งบอกปรากฏการณ์ Eutrophication

**NDCI** (Normalized Difference Chlorophyll Index)
$$NDCI = \frac{B5_{RedEdge1} - B4_{Red}}{B5_{RedEdge1} + B4_{Red}}$$

Sentinel-2 มี Red Edge bands (B5, B6, B7) เหมาะกับการวัด Chlorophyll

In [ ]:
# วิเคราะห์คลอโรฟิลล์ในทะเลสาบสงขลา
s2_songkhla = get_s2_clean(songkhla_lake, '2024-01-01', '2024-03-31', cloud_pct=30)

# NDCI = (B5 - B4) / (B5 + B4)
ndci = s2_songkhla.normalizedDifference(['B5', 'B4']).rename('NDCI')

# 2-Band Ratio (B5/B4) - อีกวิธีง่ายๆ
chl_ratio = s2_songkhla.select('B5').divide(s2_songkhla.select('B4')).rename('Chl_Ratio')

# Mask เฉพาะน้ำ
ndwi_songkhla = s2_songkhla.normalizedDifference(['B3', 'B8'])
water_mask = ndwi_songkhla.gt(0)

ndci_water = ndci.updateMask(water_mask).clip(songkhla_lake)

vis_chl = {
    'min': -0.2,
    'max': 0.3,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}

Map4 = geemap.Map(center=[7.4, 100.35], zoom=11)
Map4.addLayer(s2_songkhla.clip(songkhla_lake),
             {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500},
             'True Color')
Map4.addLayer(ndci_water, vis_chl, 'NDCI (Chlorophyll Index)')
Map4.add_colorbar(vis_chl, label='NDCI', layer_name='NDCI (Chlorophyll Index)')
Map4

---
## 6. TSS - Total Suspended Solids (ตะกอนแขวนลอย)

ประมาณค่าตะกอนแขวนลอยจากค่าสะท้อนในช่วง Red และ NIR  
TSS สูง → น้ำขุ่นจากตะกอน

In [ ]:
# TSS Proxy จาก Sentinel-2
# ใช้ B4 (Red) ที่มีความสัมพันธ์กับ TSS
# TSS ∝ Red band reflectance (simplified)

s2_gulf2 = get_s2_clean(upper_gulf, '2024-09-01', '2024-11-30')  # หน้าฝน

red_reflectance = s2_gulf2.select('B4').divide(10000)

# ประมาณค่า TSS (mg/L) - สมการเชิงประจักษ์ตัวอย่าง
# TSS = a * Red_reflectance + b (ต้อง calibrate กับข้อมูลภาคสนาม)
tss_proxy = red_reflectance.multiply(3000).rename('TSS_proxy')  # ค่าประมาณ

# Mask เฉพาะน้ำ
ndwi_gulf2 = s2_gulf2.normalizedDifference(['B3', 'B8'])
water_mask2 = ndwi_gulf2.gt(0)
tss_water = tss_proxy.updateMask(water_mask2).clip(upper_gulf)

vis_tss = {
    'min': 0,
    'max': 100,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'brown']
}

Map5 = geemap.Map(center=[13.2, 100.5], zoom=9)
Map5.addLayer(tss_water, vis_tss, 'TSS Proxy (mg/L)')
Map5.add_colorbar(vis_tss, label='TSS Proxy (mg/L)',
                  layer_name='TSS Proxy (mg/L)')
Map5

---
## 7. Sea Surface Temperature (SST)

**NOAA OISST** (Optimum Interpolation Sea Surface Temperature)  
- ความละเอียด: ~27 km (0.25°)  
- รายวัน, อัพเดตต่อเนื่อง (1981-ปัจจุบัน)  
- Band: `sst` (°C)  
- Dataset: `NOAA/CDR/OISST/V2_1`  

> ⚠️ **หมายเหตุ:** `NASA/OCEANDATA/MODIS-Terra/L3SMI` หยุดอัพเดตที่ ก.พ. 2022  
> จึงใช้ NOAA OISST แทน ซึ่งมีข้อมูลถึงปัจจุบัน

In [ ]:
# โหลด NOAA OISST (Optimum Interpolation SST)
gulf_of_thailand = ee.Geometry.Rectangle([99, 5, 105, 14])

sst = ee.ImageCollection('NOAA/CDR/OISST/V2_1') \
    .filterDate('2024-01-01', '2024-12-31') \
    .select('sst') \
    .filterBounds(gulf_of_thailand)

print(f'จำนวนภาพ OISST: {sst.size().getInfo()} ภาพ')

# SST เฉลี่ยรายเดือน
months_list = ee.List.sequence(1, 12)

def monthly_sst(month):
    month = ee.Number(month)
    monthly = sst.filter(ee.Filter.calendarRange(month, month, 'month')).mean()
    return monthly.set('month', month).set('system:time_start',
        ee.Date.fromYMD(2024, month, 1).millis())

monthly_sst_col = ee.ImageCollection.fromImages(months_list.map(monthly_sst))

# แสดง SST เดือนเมษายน (ร้อนที่สุด)
sst_april = monthly_sst_col.filter(
    ee.Filter.eq('month', 4)
).first().clip(gulf_of_thailand)

vis_sst = {
    'min': 26,
    'max': 32,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'orange', 'red']
}

Map6 = geemap.Map(center=[9.5, 102], zoom=6)
Map6.addLayer(sst_april, vis_sst, 'SST April 2024 (°C)')
Map6.add_colorbar(vis_sst, label='SST (°C)', layer_name='SST April 2024 (°C)')
Map6

In [ ]:
# กราฟ SST รายเดือนในอ่าวไทย
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def get_sst_mean(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=gulf_of_thailand,
        scale=4000,
        maxPixels=1e9
    )
    return image.set('sst_mean', stats.get('sst'))

sst_stats = monthly_sst_col.map(get_sst_mean)
sst_values = sst_stats.aggregate_array('sst_mean').getInfo()

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(month_labels, sst_values, 'ro-', linewidth=2, markersize=8)
ax.fill_between(range(len(sst_values)), sst_values, alpha=0.2, color='orange')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('SST (°C)', fontsize=12)
ax.set_title('Monthly Sea Surface Temperature - Gulf of Thailand 2024 (NOAA OISST)',
             fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## 8. วิเคราะห์แหล่งน้ำจืด: เขื่อนภูมิพล

ใช้ NDWI ติดตามพื้นที่ผิวน้ำของอ่างเก็บน้ำ

In [ ]:
# เปรียบเทียบพื้นที่น้ำของเขื่อนภูมิพล: หน้าแล้ง vs หน้าฝน

# หน้าแล้ง (เม.ย. 2024)
s2_dry = get_s2_clean(bhumibol_dam, '2024-03-01', '2024-05-31')
ndwi_dry = s2_dry.normalizedDifference(['B3', 'B8']).gt(0.3).selfMask()

# หน้าฝน (ต.ค. 2024)
s2_wet = get_s2_clean(bhumibol_dam, '2024-09-01', '2024-11-30')
ndwi_wet = s2_wet.normalizedDifference(['B3', 'B8']).gt(0.3).selfMask()

Map7 = geemap.Map(center=[17.25, 99.05], zoom=12)
Map7.addLayer(s2_dry.clip(bhumibol_dam),
             {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000},
             'True Color (Dry)')
Map7.addLayer(ndwi_dry.clip(bhumibol_dam), {'palette': ['cyan']}, 'Water - Dry Season')
Map7.addLayer(ndwi_wet.clip(bhumibol_dam), {'palette': ['blue']}, 'Water - Wet Season')
Map7

In [ ]:
# คำนวณพื้นที่ผิวน้ำในแต่ละฤดู
def calc_water_area(water_image, geometry, label):
    area = water_image.unmask(0).multiply(ee.Image.pixelArea()).divide(1e6)
    total = area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=10,
        maxPixels=1e10
    ).getInfo()
    area_km2 = list(total.values())[0] if total else 0
    print(f'{label}: {area_km2:.2f} km²')
    return area_km2

area_dry = calc_water_area(ndwi_dry, bhumibol_dam, 'Dry Season')
area_wet = calc_water_area(ndwi_wet, bhumibol_dam, 'Wet Season')
print(f'Difference: {area_wet - area_dry:.2f} km² ({((area_wet-area_dry)/area_dry*100):.1f}%)')

---
## 9. รวมทุกดัชนีคุณภาพน้ำ (Dashboard Style)

In [ ]:
# สร้างแผนที่รวมดัชนีคุณภาพน้ำ อ่าวไทยตอนบน
s2_final = get_s2_clean(upper_gulf, '2024-01-01', '2024-03-31')

# คำนวณดัชนีทั้งหมด
ndwi_final = s2_final.normalizedDifference(['B3', 'B8']).rename('NDWI')
water = ndwi_final.gt(0)

ndti_final = s2_final.normalizedDifference(['B4', 'B3']).rename('NDTI') \
    .updateMask(water).clip(upper_gulf)
ndci_final = s2_final.normalizedDifference(['B5', 'B4']).rename('NDCI') \
    .updateMask(water).clip(upper_gulf)
tss_final = s2_final.select('B4').divide(10000).multiply(3000).rename('TSS') \
    .updateMask(water).clip(upper_gulf)

# แสดงทุกดัชนี
Map8 = geemap.Map(center=[13.2, 100.5], zoom=9)

Map8.addLayer(s2_final.clip(upper_gulf),
             {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000},
             'True Color')
Map8.addLayer(ndti_final,
             {'min': -0.2, 'max': 0.3, 'palette': ['blue', 'cyan', 'green', 'yellow', 'brown']},
             'Turbidity (NDTI)', shown=False)
Map8.addLayer(ndci_final,
             {'min': -0.2, 'max': 0.3, 'palette': ['blue', 'cyan', 'green', 'yellow', 'red']},
             'Chlorophyll (NDCI)', shown=False)
Map8.addLayer(tss_final,
             {'min': 0, 'max': 100, 'palette': ['blue', 'cyan', 'yellow', 'orange', 'brown']},
             'TSS Proxy (mg/L)', shown=False)
Map8

---
## 10. Export ผลลัพธ์

In [ ]:
# Export ดัชนีคุณภาพน้ำทั้งหมดเป็น multi-band image
wq_stack = ndti_final.addBands(ndci_final).addBands(tss_final)

task = ee.batch.Export.image.toDrive(
    image=wq_stack,
    description='Water_Quality_Upper_Gulf_2024',
    folder='GEE_Workshop',
    fileNamePrefix='water_quality_gulf_2024',
    region=upper_gulf,
    scale=10,
    crs='EPSG:4326',
    maxPixels=1e10
)
task.start()
print('เริ่ม Export Water Quality Map ✅')

---
## 📝 สรุป Module 4

| ดัชนี | สูตร | ใช้วัดอะไร | ค่าที่ดี |
|-------|------|-----------|--------|
| NDWI | (Green-NIR)/(Green+NIR) | ตรวจจับน้ำ | > 0.3 = น้ำ |
| MNDWI | (Green-SWIR)/(Green+SWIR) | ตรวจจับน้ำ (เมือง) | > 0.3 = น้ำ |
| NDTI | (Red-Green)/(Red+Green) | ความขุ่น | สูง = ขุ่น |
| NDCI | (RedEdge-Red)/(RedEdge+Red) | คลอโรฟิลล์ | สูง = สาหร่ายเยอะ |
| TSS | Red reflectance × factor | ตะกอนแขวนลอย | สูง = ตะกอนเยอะ |
| SST | NOAA OISST | Sea Surface Temperature | ~27 km |

### ⚠️ ข้อควรระวัง
- ค่าดัชนีจาก Remote Sensing เป็น **proxy** ต้อง calibrate กับข้อมูลภาคสนาม
- Cloud masking สำคัญมากสำหรับการวิเคราะห์น้ำ
- Sun glint อาจทำให้ค่าผิดพลาด (โดยเฉพาะทะเล)
- Shallow water / bottom reflectance อาจส่งผลต่อค่าดัชนี

### 📚 References
- McFeeters, S.K. (1996). NDWI
- Xu, H. (2006). MNDWI
- Dogliotti, A.I. et al. (2015). Turbidity from Satellite
- Mishra, S. & Mishra, D.R. (2012). NDCI

---

## 🎉 จบ Workshop!

### สรุปทั้ง 4 Module
| Module | หัวข้อ | Datasets หลัก |
|--------|--------|---------------|
| 1 | อุตุนิยมวิทยา | CHIRPS, ERA5-Land |
| 2 | อุทกวิทยา | JRC Water, Sentinel-1, ERA5 Soil Moisture |
| 3 | ไฟป่า | MODIS Fire, FIRMS, MODIS Burned Area, Sentinel-2 dNBR |
| 4 | คุณภาพน้ำ | Sentinel-2 (NDWI/NDTI/NDCI/TSS), MODIS SST |

### 🔗 Resources
- [GEE Python API Docs](https://developers.google.com/earth-engine/guides/python_install)
- [GEE Dataset Catalog](https://developers.google.com/earth-engine/datasets)
- [geemap](https://geemap.org/)
- [GEE Community](https://gis.stackexchange.com/questions/tagged/google-earth-engine)